# 1. Introduction

In this notebook, gemma_2b is integrated with LangChain to summarize Kaggle solution write-ups.<br>
To accomplish this, the Keras model of gemma_2b is wrapped as a custom model within LangChain.<br>
I hope this aids Kagglers in incorporating modern techniques such as RAG, CoT, and so on, through LangChain.

# 2. Install packages

First of all, necesarry packages are installed as follows,

In [1]:
!pip install -q -U langchain
!pip install -q -U keras-nlp
!pip install -q -U keras>3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 

# 3. Prepare Gemma Model
## 3.1 Import Model
In this notebook, Gemma Model is used locally. Gemma model can be imported as a keras Model.

In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

2024-03-02 12:04:20.985459: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 12:04:20.985579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 12:04:21.087460: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
%%time
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


CPU times: user 6.59 s, sys: 9.26 s, total: 15.9 s
Wall time: 1min


## 3.2 Run gemma model directly
Gemma models can generate sentences as follows,

In [4]:
%%time
print(gemma_lm.generate("hi, how are you doing?", max_length=256))

hi, how are you doing?

I'm doing well, thank you for asking! I'm always happy to hear from you. How about yourself? How are you doing today?
CPU times: user 12.8 s, sys: 118 ms, total: 12.9 s
Wall time: 11.9 s


## 3.3 Define the custom model for LangChain

Since keras_nlp is not compatible with LangChain, we need to define a custom model.

The details are described on the page below.<br>
https://python.langchain.com/docs/modules/model_io/llms/custom_llm

In [5]:
from typing import Any, Optional, List, Mapping
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

class GemmaLC(LLM):

    model: Any = None
    n: int = None

    def __init__(self, keras_model, n):
        super(GemmaLC, self).__init__()
        self.model = keras_model
        self.n = n

    @property
    def _llm_type(self) -> str:
        return "Gemma"

    def _call(self, prompt: str, stop: Optional[List[str]] = None, **kwargs) -> str:

        generated = self.model.generate(prompt, max_length=self.n)

        # post-processing to extract the result of summarization
        split_string = generated.split("SUMMARY:", 1)
        if len(split_string) > 1:            
            return split_string[1].lstrip('\n')
        else:
            return generated.lstrip('\n')

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [6]:
gemma_lc = GemmaLC(gemma_lm, 1024)

# 4. Create a summarizer
## 4.1 Prompt Engneering
Since solution write-ups include many technical terms, similar to scientific papers, the default prompt tends to yield outputs like "The paper introduces ...".
To address this, I've created the following prompt that processes summarization with the consideration that the document is a part of Kaggle write-up.

In addition, we used a prompt which specifies the number of tokens to control the length of final summary.

In [7]:
from langchain_core.prompts import PromptTemplate

map_prompt_template = """Write a concise summary of the following kaggle solution writeup:


"{text}"


CONCISE SUMMARY:"""
MAP_PROMPT = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """Write a 300 letters summary of the following kaggle solution writeup:


"{text}"


300 LETTERS SUMMARY:"""
COMBINE_PROMPT = PromptTemplate(template=combine_prompt_template, input_variables=["text"])

## 4.2 summarization chain
To summarize documents using LangChain, a summarization chain needs to be defined.

The details are described on the following page:<br>
https://python.langchain.com/docs/use_cases/summarization

Make sure that the maximum token length for gemma_2b is 1024.

In [8]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(
    gemma_lc, chain_type="map_reduce",
    map_prompt=MAP_PROMPT,
    combine_prompt=COMBINE_PROMPT,
    token_max=1024
)

In [9]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


def summarize(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=50)
    texts = text_splitter.split_text(text)
    paged_docs = [Document(page_content=t) for t in texts]
    return chain.invoke(paged_docs)["output_text"]

# 5. Prepare solution write-ups

I'm following [Darek's way](https://www.kaggle.com/datasets/thedrcat/kaggle-winning-solutions-methods/) to prepare solution write-ups, which scrapes the list of solutions from [Kaggle Solutions](https://farid.one/kaggle-solutions/) and merges with kaggle-ai-report.

I found some areas for improvement to gather more solutions, so I made a few modifications as shown below.

## 5.1 Scraping

There's already a wealth of information collected on the amazing [Kaggle Solutions](https://farid.one/kaggle-solutions/) website, so let's start there - we'll scrape all the top solution description links along with the competition data. 

In [10]:
from bs4 import BeautifulSoup
import requests

url = "https://farid.one/kaggle-solutions/"

response = requests.get(url)
html = response.text

soup = BeautifulSoup(html, 'html.parser')

links = []
for li_tag in soup.find_all('li', class_='secondary'):
    link = li_tag.find('a')
    if link is None:
        continue

    if 'discussion' in link['href']:
        medal_img = li_tag.find('img')
        if medal_img is None:
            continue
        if medal_img.next_sibling is not None:
            # case:1 <img />
            place = medal_img.next_sibling.strip().split(' ')[0]
        else:
            # case:2 <img></img>
            place = list(li_tag.find('img').children)[0].strip().split(' ')[0]

        competition_name = li_tag.find_previous('b').text.split('. ')[-1].strip()

        info_tuple = (link['href'], int(place[:-2]), place, competition_name)
        links.append(info_tuple)

print(len(links))

3101


In [11]:
import pandas as pd
scraped = pd.DataFrame(data=links, columns=['link', 'place', 'place_order', 'competition_name'])
scraped.head()

,link,place,place_order,competition_name
0,https://www.kaggle.com/c/playground-series-s4e...,2,2nd,"Playground Series - Season 4, Episode 1"
1,https://www.kaggle.com/c/playground-series-s4e...,3,3rd,"Playground Series - Season 4, Episode 1"
2,https://www.kaggle.com/c/playground-series-s4e...,17,17th,"Playground Series - Season 4, Episode 1"
3,https://www.kaggle.com/c/santa-2023/discussion...,1,1st,Santa 2023 - The Polytope Permutation Puzzle
4,https://www.kaggle.com/c/santa-2023/discussion...,12,12th,Santa 2023 - The Polytope Permutation Puzzle


## 5.2 Combining with Kaggle Writeups

Kaggle doesn't like scraping discussion posts, but fortunately they provided us a bunch of writeups in the competition data. Let's merge it with the scraped data. 

In [12]:
import numpy as np
import pandas as pd

df = pd.read_csv('../input/2023-kaggle-ai-report/kaggle_writeups_20230510.csv')

scraped['nm'] = scraped.link.apply(lambda x: x.split('/')[-1])
df['nm'] = df['Writeup URL'].apply(lambda x: x.split('/')[-1])
writeup_mapping = dict(zip(df.nm, df['Writeup']))
scraped['writeup'] = scraped.nm.apply(lambda x: writeup_mapping[x] if x in writeup_mapping else np.nan)

writeups = scraped.dropna()
writeups.head()

,link,place,place_order,competition_name,nm,writeup
458,https://www.kaggle.com/c/asl-signs/discussion/...,1,1st,Google - Isolated Sign Language Recognition,406684,"<p>First of all, I would like to express my gr..."
459,https://www.kaggle.com/c/asl-signs/discussion/...,2,2nd,Google - Isolated Sign Language Recognition,406306,<h2>TLDR</h2>\n<p>We used an approach similar ...
460,https://www.kaggle.com/c/asl-signs/discussion/...,3,3rd,Google - Isolated Sign Language Recognition,406568,<p>We used an <strong>ensemble of six conv1d m...
461,https://www.kaggle.com/c/asl-signs/discussion/...,4,4th,Google - Isolated Sign Language Recognition,406673,<p>I would like to thank the organizers and al...
462,https://www.kaggle.com/c/asl-signs/discussion/...,5,5th,Google - Isolated Sign Language Recognition,406491,<p>Here is a quick overview of the 5th-place s...


## 5.3 Combining with Meta Kaggle
To add the data of teams and private/public scores, use meta-kaggle and merge with the writeup data.

In [13]:
teams = pd.read_csv('../input/meta-kaggle/Teams.csv')
teams = teams.dropna(axis=0, subset=['PrivateLeaderboardRank'])
int_columns = ['TeamLeaderId', 'PrivateLeaderboardSubmissionId', 'PrivateLeaderboardRank']
teams[int_columns] = teams[int_columns].astype(int)
teams = teams.loc[:, ['Id', 'CompetitionId', 'TeamName'] + int_columns]
teams.head()

,Id,CompetitionId,TeamName,TeamLeaderId,PrivateLeaderboardSubmissionId,PrivateLeaderboardRank
0,496,2435,team1,647,2192,83
1,497,2435,jonp,619,2182,25
2,499,2435,Bwaas,663,2184,100
3,500,2435,Thylacoleo,673,2187,23
4,501,2435,pjonesdotcda,435,2191,80


In [14]:
users = pd.read_csv('../input/meta-kaggle/Users.csv')
users = users.loc[:, ['Id', 'UserName', 'DisplayName']]
users.columns = ['UserId', 'UserName', 'DisplayName']
users.head()

,UserId,UserName,DisplayName
0,1,kaggleteam,Kaggle Team
1,368,antgoldbloom,Anthony Goldbloom
2,381,iguyon,Isabelle
3,383,davidstephan,David Stephan
4,384,gabewarren,Gabe Warren


In [15]:
competitions = pd.read_csv('../input/meta-kaggle/Competitions.csv')
competitions = competitions[['Id', 'Title']]
competitions.columns = ['CompetitionId', 'competition_name']
competitions.head()

,CompetitionId,competition_name
0,2408,Forecast Eurovision Voting
1,2435,Predict HIV Progression
2,2438,World Cup 2010 - Take on the Quants
3,2439,INFORMS Data Mining Contest 2010
4,2442,World Cup 2010 - Confidence Challenge


In [16]:
submissions = pd.read_csv('../input/meta-kaggle/Submissions.csv')
submissions = submissions[['Id', 'PublicScoreLeaderboardDisplay', 'PrivateScoreLeaderboardDisplay']]
submissions.columns = ['PrivateLeaderboardSubmissionId', 'PublicScoreLeaderboardDisplay', 'PrivateScoreLeaderboardDisplay']
submissions.head()

,PrivateLeaderboardSubmissionId,PublicScoreLeaderboardDisplay,PrivateScoreLeaderboardDisplay
0,269177,0.78117,0.76231
1,270257,0.84569,0.84469
2,271144,0.90246,0.91024
3,269508,0.90101,0.88353
4,269504,0.89230,0.87690


Loaded datasets are merged for each teams.

In [17]:
teams = pd.merge(teams, competitions, how='left', on='CompetitionId')
teams = pd.merge(teams, users, how='left', left_on='TeamLeaderId', right_on='UserId')
teams = pd.merge(teams, submissions, how='left', on='PrivateLeaderboardSubmissionId')
teams.head()

,Id,CompetitionId,TeamName,TeamLeaderId,PrivateLeaderboardSubmissionId,PrivateLeaderboardRank,competition_name,UserId,UserName,DisplayName,PublicScoreLeaderboardDisplay,PrivateScoreLeaderboardDisplay
0,496,2435,team1,647,2192,83,Predict HIV Progression,647.0,bradpennington,Brad Pennington,57.21149,56.35839
1,497,2435,jonp,619,2182,25,Predict HIV Progression,NaN,NaN,NaN,61.05770,65.60690
2,499,2435,Bwaas,663,2184,100,Predict HIV Progression,663.0,bwaas663,Bwaas,47.11539,50.00000
3,500,2435,Thylacoleo,673,2187,23,Predict HIV Progression,673.0,brucetabor,Bruce Tabor,62.50000,65.75140
4,501,2435,pjonesdotcda,435,2191,80,Predict HIV Progression,NaN,NaN,NaN,55.28850,56.50289


Finally, the writeup data is merged with the teams data.

In [18]:
writeups = pd.merge(writeups, teams, how='left',
                    left_on=['place', 'competition_name'],
                    right_on=['PrivateLeaderboardRank', 'competition_name'])
writeups.head()

,link,place,place_order,competition_name,nm,writeup,Id,CompetitionId,TeamName,TeamLeaderId,PrivateLeaderboardSubmissionId,PrivateLeaderboardRank,UserId,UserName,DisplayName,PublicScoreLeaderboardDisplay,PrivateScoreLeaderboardDisplay
0,https://www.kaggle.com/c/asl-signs/discussion/...,1,1st,Google - Isolated Sign Language Recognition,406684,"<p>First of all, I would like to express my gr...",9972071.0,46105.0,Hoyeol Sohn,5003978.0,31484928.0,1.0,5003978.0,hoyso48,hoyso48,0.810313,0.892929
1,https://www.kaggle.com/c/asl-signs/discussion/...,2,2nd,Google - Isolated Sign Language Recognition,406306,<h2>TLDR</h2>\n<p>We used an approach similar ...,9940724.0,46105.0,✌️,4212496.0,31443842.0,2.0,4212496.0,kolyaforrat,Kolya Forrat,0.812955,0.888290
2,https://www.kaggle.com/c/asl-signs/discussion/...,3,3rd,Google - Isolated Sign Language Recognition,406568,<p>We used an <strong>ensemble of six conv1d m...,9978686.0,46105.0,SabaiSabai,1187467.0,31463896.0,3.0,1187467.0,mrnnnn,Ruslan Grimov,0.804671,0.884242
3,https://www.kaggle.com/c/asl-signs/discussion/...,4,4th,Google - Isolated Sign Language Recognition,406673,<p>I would like to thank the organizers and al...,9971211.0,46105.0,ohkawa3,1630583.0,31484241.0,4.0,1630583.0,chack3,ohkawa3,0.799886,0.882355
4,https://www.kaggle.com/c/asl-signs/discussion/...,5,5th,Google - Isolated Sign Language Recognition,406491,<p>Here is a quick overview of the 5th-place s...,9963304.0,46105.0,⭐⭐⭐in the prize line⭐⭐⭐,4578277.0,31484310.0,5.0,4578277.0,yuanzhezhou,yuanzhe zhou,0.804457,0.880901


# 6. Summarize solution write-ups

Now, we can summarize kaggle solultion write-ups.

Each solution is summarized by the summarization chain.
**Overall Summary** and **Comparision Table** are also created by Gemma at the end of the procedure.

In [19]:
from IPython.display import display, Markdown


def create_overall_summary(solutions_md):
    overall_summary_md = gemma_lm.generate(f"""
Create an overall summary of the kaggle competition from the following solution writeups:

{solutions_md}

OVERALL SUMMARY:""", max_length=2048).split("SUMMARY:", 1)[1]
    overall_summary_md = "### Overall Summary" + overall_summary_md
    return overall_summary_md


def create_comparision_table(solutions_md):
    comparison_table_md = gemma_lm.generate(f"""
Create an approach comparison table by Markdown format from the following kaggle solutions (each row corresponds to each solution):

{solutions_md}

APPROACH COMPARISON TABLE:""", max_length=2048).split("TABLE:", 1)[1]
    comparison_table_md = "### Comparison Table" + comparison_table_md
    return comparison_table_md


def top_solutions(writeups, competition_name):
    '''summarize top10 solutions'''

    # filter target rows
    df = writeups.query('competition_name == @competition_name').drop_duplicates(subset=['place'])
    df = df.query('place <= 10').reset_index(inplace=False)

    solutions_md = ''
    display_solutions_md = ''
    for i, row in df.iterrows():

        solution_title = f'{row["place_order"]} place solution'
        solution_txt = summarize(row["writeup"])
        solutions_md += f'''### {solution_title}
{solution_txt}
'''

        if row["place"] <= 3:
            # display raw summaries for top 3 solutions
            display_solutions_md += f'''### [{solution_title}]({row["link"]})
#### Team: {row["TeamName"]} &nbsp; Leader: [{row["DisplayName"]}](https://www.kaggle.com/{row["UserName"]}) &nbsp; Public Score: {row["PublicScoreLeaderboardDisplay"]} &nbsp; Private Score: {row["PrivateScoreLeaderboardDisplay"]}
{solution_txt}
'''

    # display the overall summary
    overall_summary_md = create_overall_summary(solutions_md)
    display(Markdown(overall_summary_md))

    # display the comparision table
    comparison_table_md = create_comparision_table(solutions_md)
    display(Markdown(comparison_table_md))

    # display raw summaries for top 3 solutions
    display(Markdown(display_solutions_md))

Four exapmles are shown as follows,

## 6.1 OTTO – Multi-Objective Recommender System

In [20]:
%%time
top_solutions(writeups, "OTTO – Multi-Objective Recommender System")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1339 > 1024). Running this sequence through the model will result in indexing errors


### Overall Summary

The Kaggle competition showcased a diverse range of approaches to improving recommendation systems. While some solutions focused on improving features related to item2item, others addressed broader aspects such as session ranking and candidate generation. The competition also highlighted the importance of ensemble methods and the effectiveness of different similarity measures and candidate selection techniques.

### Comparison Table

| **Solution** | **Focus** | **Model** | **Feature Engineering** | **Ensemble** | **Similarity Measures** |
|---|---|---|---|---|---|
| 1 | Covisitation Matrix | Neural Networks | Candidate Generation | N/A | Co-occurrence matrices |
| 2 | Item2item | XGBoost & CatBoost | Feature Engineering | Blending | XGBoost & CatBoost |
| 3 | Landing Page Rank | XGB reranker | Feature Engineering | Ensemble | LightGBM |
| 4 | Team Collaboration | Two Teams | Different Train-Valid Splits | Ensemble | N/A |
| 5 | Candidate Set | Feature Creation & Selection | N/A | Ensemble | Lightgbm |
| 6 | Data Retrieval | Kaggle Forum | N/A | N/A | N/A |
| 7 | Image Classification | LightGBM | Co-visitation Matrix | Ensemble | N/A |
| 8 | Click Prediction | ProNE | N/A | N/A | N/A |
| 9 | Candidate Reranking | Feature Engineering | N/A | N/A | N/A |
| 10 | Cart/Order Reranking | Co-visit Features | N/A | N/A | N/A |

### [1st place solution](https://www.kaggle.com/c/otto-recommender-system/discussion/384022)
#### Team: mrkmakr &nbsp; Leader: [mrkmakr](https://www.kaggle.com/mrkmakr) &nbsp; Public Score: 0.60541 &nbsp; Private Score: 0.60503
The solution focuses on using a covisitation matrix to model the relationships between different features. It uses multiple versions of the covisitation matrix, with different weighting and aggregation periods. It also uses neural networks to predict subsequent aids and focuses on samples that are not predicted well. The solution uses different techniques to adjust the session embedding based on the prediction target aid type. Some models are trained by using only non visited aids as targets to avoid overlapping information with revisitation based candidates and features. The solution averaged the predicted scores of the rankers and tested if it was a better method than voting.
### [2nd place solution](https://www.kaggle.com/c/otto-recommender-system/discussion/382790)
#### Team: SOS3 &nbsp; Leader: [ONODERA](https://www.kaggle.com/onodera) &nbsp; Public Score: 0.60401 &nbsp; Private Score: 0.60446
The candidate focused on improving features related to item2item, including count, time difference, sequence difference, weighted above features, and aggregation of these features. They used XGBoost and CatBoost for model building and then blended the results by rank. The candidate acknowledged the contributions of cuDF and cuML and expressed gratitude to RAPIDS for their assistance.
### [3rd place solution](https://www.kaggle.com/c/otto-recommender-system/discussion/383013)
#### Team: G & B & D & T &nbsp; Leader: [Giba](https://www.kaggle.com/titericz) &nbsp; Public Score: 0.60437 &nbsp; Private Score: 0.60382
The solution focuses on improving the landing page rank (LB) of a recommender system by addressing 3 significant ideas. It uses candidate generation and co-occurrence count extraction to improve the CV and LB scores of an XGB reranker. The code creates more


CPU times: user 4min 19s, sys: 667 ms, total: 4min 20s
Wall time: 4min 23s


## 6.2 Foursquare - Location Matching

In [21]:
%%time
top_solutions(writeups, "Foursquare - Location Matching")

### Overall Summary

The Kaggle competition showcased diverse approaches to various natural language processing (NLP) tasks. While some solutions relied heavily on feature engineering and ensemble methods, others employed more advanced techniques such as BERT, graph neural networks, and multi-lingual representation. The competition also highlighted the importance of optimizing model hyperparameters and selecting appropriate feature sets for optimal performance.

### Comparison Table

| **Solution** | **Focus** | **Candidate Generation** | **Feature Engineering** | **Model** | **Evaluation** |
|---|---|---|---|---|---|
| 1 | Generating new candidates | Cross-validation with 2 folds | Feature selection | LightGBM and Forestinference | F1 score |
| 2 | Matching locations | K-nearest neighbors | String similarity measures | SVM | Accuracy |
| 3 | Text classification | NLP deep learning | Bi-encoder models |  | F1 score |
| 4 | Sentiment analysis | BERT model | Feature engineering |  | Accuracy |
| 6 | Generating candidate data | Graph probability convolution |  | CatBoost |  |  |
| 7 | Pre-processing and generating candidates | Various techniques | LightGBM |  |  |
| 8 | Multi-lingual text representation |  |  | LightGBM |  |  |
| 9 | Selecting candidate features | Two-fold cross-validation | Feature engineering and post-processing | CatBoost | F1 score |

### [1st place solution](https://www.kaggle.com/c/foursquare-location-matching/discussion/336055)
#### Team: re:waiwai &nbsp; Leader: [Takoi](https://www.kaggle.com/takoihiraokazu) &nbsp; Public Score: 0.97808 &nbsp; Private Score: 0.97768
The Kaggle solution aimed to generate new candidates by dividing the process into four stages: creating candidates, feature engineering and LightGBM, ensemble methods, and post-processing. However, due to leakage, the team had to spend significant time searching for leaks in the final stages.

The solution used a cross-validation strategy with 2 folds to evaluate the performance of different feature selection methods. The top 20 candidates were selected for each feature selection method, and the LightGBM and Forestinference algorithms were used to make predictions.

The solution focused on building a predictive model for Foursquare lightgbm baseline using different transformer models with different feature sets. The results showed that merging the training and validation data (1 + 2) generally led to the best performance, while merging with the test data (1 + 2 + 3) was less effective.
### [2nd place solution](https://www.kaggle.com/c/foursquare-location-matching/discussion/336090)
#### Team: 2:30 &nbsp; Leader: [T0m](https://www.kaggle.com/tomyanabe) &nbsp; Public Score: 0.97285 &nbsp; Private Score: 0.9726
This script focuses on the task of matching locations in a dataset of foursquare location coordinates. It employs a two-step approach to achieve this: candidate generation and binary classification. The candidate generation step utilizes a k-nearest neighbors approach to identify potential matches between locations, while the binary classification step utilizes a support vector machine (SVM) to determine whether the matches are valid or not.

The script uses various string similarity measures and token-based approaches to address the task of modeling the similarity between names and categories. It employs
### [3rd place solution](https://www.kaggle.com/c/foursquare-location-matching/discussion/338112)
#### Team: Psi &nbsp; Leader: [Psi](https://www.kaggle.com/philippsinger) &nbsp; Public Score: 0.96866 &nbsp; Private Score: 0.96847
The solution is based on a two-stage approach using NLP deep learning models. The first stage uses an ArcFace model to predict the point-of-interest of a record and put similar records into a similar embedding space. The second stage trains a bi-encoder model based on the proposed candidates from the first stage. The solution uses various bi-encoder models to train for TP/FP of pairs. The training data consists of records where each record represents a pair of entities. The solution uses cross-validation to select the best set of candidates to train on.


CPU times: user 2min 47s, sys: 122 ms, total: 2min 47s
Wall time: 2min 47s


## 6.3 G2Net Detecting Continuous Gravitational Waves

In [22]:
%%time
top_solutions(writeups, "G2Net Detecting Continuous Gravitational Waves")

### Overall Summary

The Kaggle competition focused on deep learning and creative denoising. The solutions explored different approaches to address the challenges of gravitational wave detection, including noise reduction, signal enhancement, and parameter estimation. The best solution achieved an accuracy of 0.855/0.849, demonstrating the effectiveness of deep learning techniques for gravitational wave detection.

### Comparison Table

| **Solution** | **Focus** | **Noise Removal** | **Signal Generation** | **Model Architecture** | **Loss Function** |
|---|---|---|---|---|---|
| 1 | Signal extraction | Random search | Simulated data | 1D CNN | AUC ROC |
| 2 | Signal enhancement | None | Real data | None | Mean score |
| 3 | Deep learning | None | Real data | CNNs | Mean score |
| 4 | Candidate CW parameter generation | Dynamic programming | None | None | AUC ROC |
| 5 | Signal analysis | None | Real data | Logistic function | None |
| 6 | Waveform analysis | None | Real data | SA | kth percentile |
| 7 | Matched filter | Synthetic and pseudo-labeled data | None | CNN | Precision, AUC |
| 8 | Matching filter | Synthetic and pseudo-labeled data | None | CNN | Precision, AUC |
| 9 | Data augmentation | Gaussian noise | Time-varying Gaussian noise | CNN | Mean, standard deviation |
| 10 | CNN with normalization | None | Time-varying Gaussian noise | CNN | RobustScaler, multitask learning |

### [1st place solution](https://www.kaggle.com/c/g2net-detecting-continuous-gravitational-waves/discussion/375910)
#### Team: 🐢 Jun Koda &nbsp; Leader: [🐢 Jun Koda](https://www.kaggle.com/junkoda) &nbsp; Public Score: 0.84866 &nbsp; Private Score: 0.86376
The author attempted to build a deep neural network model to detect gravitational waves from the Earth's rotation. Despite using 1-dimensional convolutional neural networks, they failed to achieve the same accuracy as the top-performing solutions. The solution focuses on calculating the maximum power of the signal patterns by extracting the power from the simulations and then summing the weighted power along the frequency axis. The solution addresses the issue of highly skewed values due to the large number of templates and frequencies by employing several normalization techniques. The solution uses a sinc kernel to collect the signal and recompute the power sum with this kernel around the largest-power line. It then applies a sigmoid to the standardized power sum and submits the result.
### [2nd place solution](https://www.kaggle.com/c/g2net-detecting-continuous-gravitational-waves/discussion/376504)
#### Team: PreferredWave &nbsp; Leader: [charmq](https://www.kaggle.com/charmq) &nbsp; Public Score: 0.84992 &nbsp; Private Score: 0.85545
The Kaggle solution focused on identifying and removing noise from a dataset to improve the performance of a machine learning model. By identifying and removing noise, the model was able to achieve a higher accuracy of 0.855/0.849 (1st in the public LB). The solution uses random search to find the optimal combination of parameters that maximizes the mean power of the signal. The best settings are then used to calculate the mean score of the two detectors for each data.
### [3rd place solution](https://www.kaggle.com/c/g2net-detecting-continuous-gravitational-waves/discussion/376233)
#### Team: BearWaves (not prize eligible) &nbsp; Leader: [RabotniKuma](https://www.kaggle.com/analokamus) &nbsp; Public Score: 0.80717 &nbsp; Private Score: 0.82634
The Kaggle competition focuses on deep learning and creative denoising. The solution focuses on generating realistic noise patterns for a fixed and infinite dataset approach. It explores the distribution of background noise and generates noise with similar statistics to the real data. Additionally, it generates signal templates with frequency dependency and injects them into the generated noise during training. The model uses CNNs with various modifications to improve the spectrogram image classification task.


CPU times: user 2min 36s, sys: 137 ms, total: 2min 36s
Wall time: 2min 36s


# 7. For further improvement

LangChain can summarize the input solution write-ups, but there seems to be room for improvement.

Upon consideration, we can suggest the following improvements:

* Using larger models
* Adjusting system prompts
* Adding preprocessing and postprocessing
* Optimizing chunk size
* Utilizing HTML tags (document structures) in write-ups

Thank you for reading this to the end. Please don't forget to upvote :)